# Recurrent neural networks for machine translation

In this script, we work on sequence to sequence (seq2seq) learning using recurrent neural networks (it's one example of "many-to-many" RNNs seen in the course). This task consists in producing one sequence of data from another (of possibly different lengths).

More specifically, we will work with textual data for the *machine translation* task: the goal is to automatically translate a sentence from one language to another.

<img src="https://pytorch.org/tutorials/_images/seq2seq.png" width="500"/>
<center><a href="https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html">Source</a></center>

**Note**: This notebook is based on [this tutorial](https://github.com/bentrevett/pytorch-seq2seq), which you are strongly encouraged to check as it goes into much more details about seq2seq models.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import math
import time

# We'll be using torchtext and spacy to do most of the pre-processing
import spacy
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

# Set a random seed for reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## Pre-processing: preparing the data

### Tokenizers

The first step is to define *tokenizers*, that is, how a string is transformed into a sequence of words (or *tokens*). For instance, "Welcome to the U.S.A.!" should be transformed into \["Welcome", "to", "the", "U.S.A.", "!"\]. These are called tokens in the general case because "!" is not a word.

We will also use language models, in order to preserve some specific rules for tokenization: with a naïve approach, "U.S.A." would be split into 6 tokens \["U", ".", "S', ".", "A", "."\], but we want to consider it as a single token.

In [ ]:
# load the German and English specific pipelines
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

# define tokenizers
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

# check on an example
print('Welcome to the U.S.A!')
print(tokenize_en('Welcome to the U.S.A.!'))

In [ ]:
# We also define 'Fields', which handle how the data will be processed. In addition to tokenizing, it can convert
# all characters to lower case and add extra tokens for the start and end of sentences.
SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

### Dataset

We use the Multi30k dataset, which contains sentences in German and English (as well as French, but it's not available using torchtext).

In [ ]:
# Load the full dataset
train_data, valid_data, test_data = Multi30k.splits(root='data/', exts = ('.de', '.en'), fields = (SRC, TRG))

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

# We take a subset of the full dataset for speed
train_data.examples = train_data.examples[:1000]
valid_data.examples = valid_data.examples[:100]
test_data.examples = train_data.examples[:100]

# Print one example
print(train_data.examples[0].src)
print(train_data.examples[0].trg)

### Vocabulary

We now use the ```build_vocab``` method of the ```Field``` to create a vocabulary from the data. A vocabulary maps each token to an integer, and using the Field also transforms it into a torch tensor.

In [ ]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

In [ ]:
# We can use the 'vocab.itos' method to convert indices to the corresponding tokens
# The first tokens in the vocabulary are 'unknown word', 'padding', 'start of sentence' and 'end of sequence'.
# Then the tokens are ranked by frequency of appearence in the dataset.
for i in range(20):
    print(TRG.vocab.itos[i])

### Dataloader

The final step is to create a dataloader to generate batches of data. Instead of using the classic ```Dataloader``` function, we use ```BucketIterator```, which creates batches by assembling sentences of similar lengths This reduces the amount of padding and therefore of useless calculation.

In [ ]:
batch_size = 128

train_dataloader, _, test_dataloader = BucketIterator.splits((train_data, valid_data, test_data),
                                                           batch_size = batch_size)

# Fetch one batch as an example
example_batch = next(iter(train_dataloader))

# Each batch contains a 'src' and 'trg' entries (source and target), corresponding to English and German sentences.
print(example_batch.src[:,1])

# The shape of this tensor should be [seq_length, batch_size] where seq_length is the maximum length of a sentence in this batch
print(example_batch.src.shape)

In [ ]:
# TO DO: write a function that takes a list of integers (such as a slice of 'example_batch' above) as input
# and return the corresponding tokens (hint: use the 'vocab.itos' method), for both English and German


## Recurrent networks basics

Now that the data is ready, let's see the basic operations used in RNNs: [embedding layers](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html?highlight=embedding#torch.nn.Embedding), [dropout](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html), and [recurrent layers](https://pytorch.org/docs/stable/nn.html#recurrent-layers).

### Embedding layer

Sentences have been tokenized and tokens have been transformed into integers. We need to further transform these integers into word vectors: the idea is that two *similar* words should have similar word vectors. 

<img src="https://ruder.io/content/images/size/w2000/2016/04/word_embeddings_colah.png" width="500"/>
<center><a href="https://ruder.io/word-embeddings-1/">Source</a></center>

This notion of similarity (and what word vectors exactly represent) is hard to define explicitly. Then, we use an embedding layer to produce these word vectors, and this layer is learned during training.
Many pre-trained word embeddings are available (e.g., word2vec) but here we will learn it from scratch along with the rest of the network.

In [ ]:
# Create an embedding layer. We need to specify:
# - the input dimension, that is, how many words in the vocabulary
# - the embedding dimension, that is, how "big" is the word vectors space 
input_dim = len(SRC.vocab)
embedding_dim = 32
src_emb_layer = nn.Embedding(input_dim, embedding_dim)

# Apply it to the example batch and display it
embedded_batch = src_emb_layer(example_batch.src)
print(embedded_batch)

# The shape of the word vectors for a batch should be [seq_length, batch_size, embedding_dim]
print(embedded_batch.shape)

### Dropout

The core idea of a dropout layer is to reduce the risk of overfitting by randomly setting some inputed values at 0. Since the non-zero inputs (and the corresponding weights in the network) are not the same from one batch to another, it results in forcing these weights not to be batch-specific, and therefore avoid overfitting.

In [ ]:
# the percentage of zeroed values (expressed between 0 and 1) is given as input
dropout_layer = nn.Dropout(0.5)
drop_batch = dropout_layer(embedded_batch)

# in this example, half of the entries (50%) are set at 0
print(drop_batch)

### Recurrent layers

<img src="https://www.researchgate.net/profile/Rezzy-Caraka/publication/346410173/figure/fig2/AS:962598073823234@1606512673418/Network-Structure-of-RNN-LSTM-and-GRU.png" width="500"/>
<center><a href="https://www.researchgate.net/profile/Rezzy-Caraka/publication/346410173_Employing_Long_Short-Term_Memory_and_Facebook_Prophet_Model_in_Air_Temperature_Forecasting/links/60077104a6fdccdcb868957f/Employing-Long-Short-Term-Memory-and-Facebook-Prophet-Model-in-Air-Temperature-Forecasting.pdf">Source</a></center>

We now see the 3 main recurrent layers (simple RNN, LSTM and GRU). We won't focused on the technical difference between these, but you can find more info online (e.g., [here](https://medium.com/analytics-vidhya/rnn-vs-gru-vs-lstm-863b0b7b1573)).

#### Simple RNN

First, let's see the basic RNN. We note $x_t$ the $t$-th element of the input to the RNN (in our case: this is the embedding after dropout). We have $h_t = \text{RNN}(x_t, h_{t-1})$, where $h_{t}$ is the hidden state. To define such an RNN in Pytorch (using ```nn.RNN```), you need to specify:

- the size of the input (here, it's the size of the embeddings)
- the size of the hidden space (hidden_dim)
- the number of layers (n_layers)

By default, the RNN is uni-directional, uses bias, and uses tanh as activation function. If you use a multi-layer RNN, you can also add dropout in the intermediate layers. You can change these by playing with the parameters of the function (see the [doc](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html#torch.nn.RNN) for more info).

**Note**: for the first element of the sequence, we have $h_1 = \text{RNN}(x_1, h_{0})$, so we normally need to provide an initial hidden state $h_0$. In pytorch, we can either provide it explicitly or not. If we don't, it will use $h_0=0$ by default. This is what we do here, and it also applies to other recurrent units (LSTM and GRU).

In [ ]:
# Define a basic RNN
hidden_dim = 50
n_layers = 2
rnn = nn.RNN(embedding_dim, hidden_dim, n_layers)

# Apply the RNN: you should give as input:
# - the embeddeding after dropout calculated beforehand 'drop_batch'
# The RNN returns:
# - 'rnn_out': the whole sequence of hidden states of the last layer
# - 'rnn_hidden': the final hidden states (for the last token) for all layers
rnn_out, rnn_hidden = rnn(drop_batch)

# Get the shape of the 'rnn_out': it should be [seq_length, batch_size, hidden_dim]
print(rnn_out.shape)

# Get the shape of the final hidden state 'rnn_hidden': it should be [n_layers, batch_size, hidden_dim]
print(rnn_hidden.shape)

In [ ]:
# TO DO: write a 3-layer bidirectional RNN (check the doc!) and apply it to the same input ('embedded_batch')
# Print the shape of rnn_out (it should be [seq_length, batch_size, 2*hidden_dim])
# Print the shape of the final hidden state (it should be [2*n_layers, batch_size, hidden_dim])
# (the factor '2' in the shapes comes from the fact that the network is bidirectional)


### LSTM

The basic RNN suffers from the gradient vanishing problem, so we will instead use a variant of it called *long short-term memory* (LSTM) networks. The key idea of LSTM is that it has an extra hidden feature called a *cell state* which allows the network to "remember" which part of the input sequence is useful or not, and therefore to avoid backpropagating the gradient throughout the whole sequence, thus avoiding gradient vanishing.

The formula for the [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) is therefore: $(h_t, c_t) = \text{LSTM}(x_t, h_{t-1}, c_{t-1})$ where $c_t$ is this extra cell state.

In [ ]:
# Define an LSTM
hidden_dim = 50
n_layers = 2
lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers)

# Apply the LSTM to the embedded batch
lstm_out, (lstm_hidden, lstm_cell) = lstm(embedded_batch)

# The shape of the output and final hidden state are the same as before.
# The final cell state as the same size as the final hidden state.
print(lstm_out.shape)
print(lstm_hidden.shape)
print(lstm_cell.shape)

### GRU

The last main type of recurrent layer is the gated reccurent unit (GRU). You can see it as sort of a simplified LSTM: it also has some memory mechanism to avoid gradient vanishing, but it outputs only a single hidden state vector (instead of the additional cell state in LSTM). It generally performs similarly with LSTM (but this depends on applications). [Writting a GRU in pytorch](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html#torch.nn.GRU) is similar to a basic RNN.

In [ ]:
# TO DO: using the doc, write a GRU layer with a hidden size of 50 and 2 layers.
# Apply it to the embedded batch as before, and print the shape of the output and final hidden state.


## Building the translation model

We'll now build the machine translation model. This model is based on two part:

- an *encoder*, which takes as input the source sentence (in German) and encodes it into a *context* vector. This context vector is sort of a summary of the whole input sentence.
- a *decoder*, which takes as input this context vector and sequentially generates a sentence in English. It always starts with the `<sos>` token and uses the context vector to generate the second token. Then, it recursively uses the last produced token and the hidden state to generate the next token.

<img src="https://github.com/bentrevett/pytorch-seq2seq/raw/49df8404d938a6edbf729876405558cc2c2b3013//assets/seq2seq1.png" />
<center><a href="https://github.com/bentrevett/pytorch-seq2seq">Source</a></center>
<center>Note: On this picture, $h_t$ represent the hidden states of the encoder and $s_t$ the hidden states of the decoder. For LSTM, we also need to consider the cell states, but they're not displayed here for brevity.</center>

### Encoder

First, we write the encoder. It consists of:

- an embedding layer to transform token indices into word vectors.
- a dropout layer to reduce overfitting.
- a 2-layer LSTM, to learn the context vector.

**Note**: For the encoder, we don't need to keep track of all the hidden states ($h_1$, $h_2$, $h_3$, etc.), we only need the final hidden state called the context vector (and denoted $z$ on the picture). Therefore, we can simply apply our LSTM on the whole sequence, instead of writting a loop explicitly (this will not be the case for the decoder).

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim_enc, n_layers, dropout_rate):
        super().__init__()
        
        # TO DO: store the parameters (reminder: use 'self.')
        
        # TO DO: create the embedding layer (transform indices into word vectors)
        
        # TO DO: create the dropout layer
        
        # TO DO: create the LSTM layer
        
        
    def forward(self, src):
        
        # TO DO: first compute the embeddings
        
        # TO DO: apply dropout to the word embeddings
        
        # TO DO: apply the LSTM layer
        
        # TO DO: return the final hidden and cell states
        return

In [ ]:
# Define the encoder parameters
input_dim = len(SRC.vocab)
embedding_dim_enc = 32
hidden_dim_enc = 50
n_layers = 1
dropout_rate = 0.5

# Instanciate the encoder and print the number of trainable parameters
lstm_encoder = Encoder(input_dim, embedding_dim_enc, hidden_dim_enc, n_layers, dropout_rate)
print('Number of parameters:', sum(p.numel() for p in lstm_encoder.parameters() if p.requires_grad))

# Apply it to the example batch
enc_hidden, enc_cell = lstm_encoder(example_batch.src)
print(enc_hidden.shape)
print(enc_cell.shape)

### Decoder

Next, we'll build our decoder. Let us note that we treat machine translation as a classification task: the decoder tries to predict the probability of all token indices in the output (target) vocabulary from an input token index. This has two implications:

- In addition to the embedding, dropout and LSTM layers, the decoder applies an extra linear layer/MLP to perform prediction of the probabilities. Therefore, this linear layer goes from a space of size `hidden_dim_dec` to a space of size `output_dim`, which is the number of tokens in the target vocabulary.
- the decoder doesn't process all the sentence at once, but token by token, since the input at step $t$ is the word that has been predicted at step $t-1$ (not just the hidden state). Therefore, the input to the decoder has a sequence length of 1 (the recursive calculation over the whole sentence will be done in the full model).

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim_dec, n_layers, dropout_rate):
        super().__init__()
        
        # TO DO: store parameters
        
        # TO DO: create the embedding, dropout and LSTM layers
        
        # TO DO: create the linear layer (it goes from a space of size 'hidden_dim_dec' to 'output_dim')
        
    def forward(self, input_idx, input_hidden, input_cell):
        
        # apply the embedding and dropout layers
        y = self.dropout_layer(self.embedding_layer(input_idx))
        
        # since y has a shape [batch_size, hidden_dim_dec], we need to unsqueeze it
        # to create an artificial 'seq_length' (=1) dimension
        y = y.unsqueeze(0)
        
        # TO DO: apply the lstm layer. Unlike the encoder, we need to store all the outputs to predict the target token
        
        # TO DO: squeeze 'output' (to remove the useless dimension 'seq_length'=1)
        
        # TO DO: apply the linear layer to predict the probabilities
        
        # TO DO: return the predicted probabilities per token, and the hidden / cell states of the LSTM decoder
        return 

In [ ]:
# Define the decoder parameters
output_dim = len(TRG.vocab)
hidden_dim_dec = 50
embedding_dim_dec = 32

# Instanciate the decoder and print the number of trainable parameters
lstm_decoder = Decoder(output_dim, embedding_dim_dec, hidden_dim_dec, n_layers, dropout_rate)
print('Number of parameters:', sum(p.numel() for p in lstm_decoder.parameters() if p.requires_grad))

# Create an artificial numerized input token (input indices) with value '2'
# (it corresponds to the 'start of sentence' or '<sos>' token)
input_idx = torch.ones(batch_size).int() * 2

# Apply the decoder
pred_proba, dec_hidden, dec_cell = lstm_decoder(input_idx, enc_hidden, enc_cell)
print(pred_proba.shape)
print(dec_hidden.shape)
print(dec_cell.shape)

In [ ]:
# TO DO: from the predicted probabilities calculated before 'pred_proba', get the index with the highest probability

# TO DO: for each element in the batch, transform this index back to an actual token (word)


### Full model

Finally, we need to implement the overall model, which takes an input sentence, produces the context vectors using the encoder, and produces the output sentence recursively using the decoder.

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        
        # Store the encoder, decoder, and the target vocabulary size
        self.encoder = encoder
        self.decoder = decoder
        self.trg_vocab_size = decoder.output_dim
        
    def forward(self, src, trg_len):
        
        # The maximum length of the target sentences in the batch is given as input 'trg_len'
        
        # Create a tensor to store the predicted probabilities from the decoder
        batch_size = src.shape[-1]
        pred_probas = torch.zeros(trg_len, batch_size, self.trg_vocab_size)
        
        # Give a probability of 1 to the element correponding to <sos> (index=2) for the first element of all sequences in the batch 
        pred_probas[0, :, 2] = 1

        # Create the first input to the decoder is the <sos> token (coded by '2' in our vocabulary)
        input_idx = torch.ones(batch_size).int() * 2
        
        # TO DO: apply the encoder to the src sentence and get the last hidden and cell states (=context vectors)

        
        # loop over tokens: it should start from 1 (not 0) since the very first token is already known (<sos>)
        for t in range(1, trg_len):

            # TO DO: apply the decoder using the current input_idx, hidden and cell states
            # It should output the predicted proba at step t (let's call it 'pred_proba_t',
            # and the updated hidden and cell states
            
            # TO DO: store this current pred_proba_t in the tensor pred_probas
            # It will be used to compute the classification loss during training
            
            # TO DO: from 'pred_proba_t', get the index with the highest probability and use it as the next input
        
        return pred_probas

In [ ]:
# Instanciate the model and print the number of parameters (should be exactly params(encoder)+params(decoder))
model = Seq2Seq(lstm_encoder, lstm_decoder)
print('Number of parameters:', sum(p.numel() for p in model.parameters() if p.requires_grad))

# TO DO: Apply the model to the 'example_batch'

# TO DO: Get the indices of highest predicted 

# TO DO: display one of the predicted sentences


## Training

Now we have our model implemented, we can train it.

In [ ]:
# The training function is very similar to what has been done before, except a few manipulation before computing the loss
def training_seq2seq(model, train_dataloader, num_epochs, loss_fn, optimizer, device='cpu', verbose=True):

    # Set the model in 'training' mode
    model.train()

    # Copy the model to device
    model.to(device)

    # Initialize a list to save the training loss over epochs
    loss_total = []

    for epoch in range(num_epochs):

        loss_current_epoch = 0

        # loop over batches
        for i, batch in enumerate(train_dataloader):

            # Get the source and target sentence, and the target length, copy it to device
            src, trg = batch.src.to(device), batch.trg.to(device)
            trg_len = trg.shape[0]

            # Set the gradients at 0
            optimizer.zero_grad()

            # Apply the model
            pred_probas = model(src, trg_len)

            # Remove the first token (always <sos>) to compute the loss
            output_dim = pred_probas.shape[-1]
            pred_probas = pred_probas[1:]

            # Reshape the pred_probas and target so that they have appropriate shapes:
            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]
            pred_probas = pred_probas.view(-1, output_dim)
            trg = trg[1:].view(-1)

            # Compute the loss and the gradients
            loss = loss_fn(pred_probas, trg)
            loss.backward()

            # backpropagation (clip the gradients at a maximum value 1 to avoid too large steps)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # Record the loss
            loss_current_epoch += loss.item()

        # At the end of each epoch, save the average loss over batches and display it
        loss_total.append(loss_current_epoch)
        if verbose:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss_current_epoch))

        # Save the trained over epochs (in case it's interrupted for some reason)
        torch.save(model.state_dict(), 'lstm_model.pt')
        
    return loss_total

In [ ]:
# Define the optimizer (ADAM), set the number of epochs
optimizer = optim.Adam(model.parameters())
num_epochs = 10

# For the loss function, since we treat the problem as a classification task, we use the cross entropy.
# We also tell it to ignore the index of the <pad> token for computation speed
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
loss_fn = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

# Apply the training function
loss_total = training_seq2seq(model, train_dataloader, num_epochs, loss_fn, optimizer)

## Evaluation

Now the model is trained, we can evaluate it.

In [ ]:
# The evaluation function is similar to what was done in previous labs.
def evaluate_seq2seq(model, eval_dataloader, loss_fn, device='cpu', verbose=True):

    # Set the model in 'eval' mode (disable dropout layer)
    model.eval()

    # Copy the model to device
    model.to(device)

    # Initialize the eval loss
    loss_eval = 0

    # loop over batches
    for i, batch in enumerate(eval_dataloader):

        # Get the source and target sentence, and the target length, copy it to device
        src, trg = batch.src.to(device), batch.trg.to(device)
        trg_len = trg.shape[0]

        # Apply the model
        pred_probas = model(src, trg_len)

        # Remove the first token (always <sos>) to compute the loss
        output_dim = pred_probas.shape[-1]
        pred_probas = pred_probas[1:]

        # Reshape the pred_probas and target so that they have appropriate shapes:
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        pred_probas = pred_probas.view(-1, output_dim)
        trg = trg[1:].view(-1)

        # Compute the loss
        loss = loss_fn(pred_probas, trg)

        # Record the loss
        loss_eval += loss.item()

    return loss_eval

In [ ]:
# TO DO: Instanciate a model and load the trained parameters

# TO DO: Evalute it on the test set and display the loss


In [ ]:
# Get some examples (source and target) in the test set
example_batch_src, example_batch_trg = example_batch.src, example_batch.trg

# TO DO: Compute predictions with the model

# TO DO: Print the true and predicted target sentences


As you can see on the example above, the results are not very good. There are many strategies to improve performance:

- *skip-filtering*, which means feeding each RNN cell with the whole context vector instead of just the previous hidden state (done [here](https://github.com/bentrevett/pytorch-seq2seq/blob/master/2%20-%20Learning%20Phrase%20Representations%20using%20RNN%20Encoder-Decoder%20for%20Statistical%20Machine%20Translation.ipynb)).
- *teacher forcing* in the decoder at training, which means using the ground truth token as input to each decoder cell instead of the predicted token from the previous cell (done [here](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb)).
- *packed padded sentences* with masking, which allows to skip the `<pad>` token in the encoder and save time (done [here](https://github.com/bentrevett/pytorch-seq2seq/blob/master/4%20-%20Packed%20Padded%20Sequences%2C%20Masking%2C%20Inference%20and%20BLEU.ipynb)).

There are also more simple changes we can do readily. These are implemented in the next script (bonus work).